In [1]:
import numpy as np
import random
from Contenedor import RandomPoblation,Bin
from BPGen import UDLPROBLEM
from PackingH import DBLF2,DBLF
import time


In [2]:
class Problem:
    def __init__(self, problem:list, id:int, algorithm:int):
        self.problem = problem
        self.id =id
        self.algorithm  = algorithm

In [3]:
def CalculateFi(pobl:list,boxes:list[list[int]],cont:list[int])->list[list[float,list[int]]]:
    ffs=[]
    for ff in pobl:
        bin=Bin(cont,len(boxes))
        DBLF2(bin=bin,itemsToPack= ff,ITEMSDATA= boxes)
        fi =(bin.getLoadVol()/np.prod(cont))**bin.getN()
        ffs.append([fi,ff])
    return ffs
def SingleFi(firefly:list[float,list[int]],boxes,cont):
    bin=Bin(cont,len(boxes))
    DBLF2(bin,firefly[1],boxes)
    fi = (bin.getLoadVol()/np.prod(cont))*bin.getN()
    firefly[0] = fi


In [4]:
class DFFA:
    def __init__(self):
        self.pob:list[list[int]] = None
    def CreatPoblation(self,num_pop:int,discrete_data:list):
        self.pob =RandomPoblation(num_pop, np.array(discrete_data))
    def Hamming(self,f1,f2):
        count =0
        n = len(f1)
        for i in np.arange(n):
            if f1[i] != f2[i]:
                count +=1
        return count
    def Intensity(self,f1:list[float,list[int]],f2:list[float,list[int]],gamma:float):
        if f1 == f2:
            return f1[0]
        dist = self.Hamming(f1[1],f2[1])
        return f1[0]/(1+gamma*(dist**2))
    def BettaStep(self,f1:list[int],f2:list[int],gamma:float)->list[int]:
        n = len(f1)
        res = [None for i in np.arange(n) ]
        visit = [False for i in np.arange(n+1)]
        noneNumber = n
        for i in np.arange(n):
            if f1[i] == f2[i]:
                res[i] = f1[i]
                visit[f1[i]] = True
        betta = 1/(1+gamma*(self.Hamming(f1,f2)**2))
        for i in np.arange(n):
            if noneNumber == 0:
                break
            if res[i] is None:
                pr = random.random()
                if pr < betta:
                    for j in range(n):
                        if not visit[f2[j]]:
                            res[i]=f2[j]
                            visit[f2[j]]=True
                            break
                else:
                     for j in range(n):
                        if not visit[f1[j]]:
                            res[i]=f1[j]
                            visit[f1[j]]=True
                            break
        return res
    def alphaStep(self,alpha:float,fi:list[int])->None:
        n = len(fi)
        for i in np.arange(n):
            prev = fi[i]
            num = round(fi[i]+alpha*(random.random()-0.5))
            if num < 1:
                num=1
            elif n < num:
                num=n 
            fi[i] = num 
            for j in np.arange(n):
                if fi[i]==fi[j] and i!=j:
                    fi[j]=prev
                    break
    def RandomMov(self,firefly:list,alpha:float,DataBoxes,BinData):
        org = firefly.copy()
        self.alphaStep(alpha,firefly[1])
        SingleFi(firefly,DataBoxes,BinData)
        if org[0]>firefly[0]:
           firefly=org
    def train(self,num_pop,gamma,Maxitr:int,DataBoxes:list[list[int]],BinData:list[int]):
        self.pob =RandomPoblation(num_pop, np.array(DataBoxes))
        n = len(DataBoxes)
        fireflyPob = CalculateFi(self.pob,DataBoxes,BinData)
        fireflyPob.sort(key=lambda x:x[0],reverse=True)
        for _ in np.arange(Maxitr):
            print(_)
            alpha = np.floor(n-((_)/Maxitr)*(n))
            fnum = len(fireflyPob)
            for i in np.arange(fnum-1):
                for j in np.arange(i+1,fnum):
                    Ii = self.Intensity(fireflyPob[i],fireflyPob[j],gamma)
                    Ij = self.Intensity(fireflyPob[j],fireflyPob[i],gamma) 
                    if Ii > Ij:
                        fireflyPob[j][1] = self.BettaStep(fireflyPob[j][1],fireflyPob[i][1],gamma)
                        self.RandomMov(firefly=fireflyPob[j],alpha=alpha,DataBoxes=DataBoxes,BinData=BinData)
            self.RandomMov(firefly=fireflyPob[0],alpha=alpha,DataBoxes=DataBoxes,BinData=BinData)
            fireflyPob.sort(key=lambda x:x[0],reverse=True)
        return fireflyPob[0]

In [5]:
contenedor = UDLPROBLEM["UDL2"]["dimension"]
datos = UDLPROBLEM["UDL2"]["24boxes"][2]
gamma=0.0004
ffly = DFFA()
bestff = ffly.train(num_pop=50,Maxitr=500,gamma=gamma,DataBoxes=datos,BinData=contenedor)

KeyboardInterrupt: 

In [ ]:
from DataLecture import ExpotarJSON
bin = Bin(contenedor,len(datos))
DBLF2(bin,bestff[1],datos)
print(bin.getN(),len(datos))
print(bin.getLoadVol()/np.prod(contenedor))
ExpotarJSON("C:/Users/nicoo/BPP/Assets/bbp.json",bin.getBoxes(),bin.dimensions)

In [ ]:
print('Num bjeto emp. ', bin.getN(),' num total: ',len(datos))
print('chromosome: ', bestff[1])
print("solution" ,[ item.id for item in bin.getBoxes() ])
print('Fi: ', bin.getLoadVol()/np.prod(contenedor))